In [13]:
import os

os.chdir("/home/handb/GeoSTHN")

In [14]:
import json
import re


def _extract_first_float(s):
    if not s:
        return None
    match = re.search(r"(\d+\.\d+)", s.strip())
    return float(match.group(1)) if match else None


def calculate_average_test_mrr(json_file_path, s, e):
    """
    计算JSON结果文件中 test mrr, test auroc, test auprc 的平均值和统计信息

    Args:
        json_file_path: JSON结果文件路径
        s: 起始索引（可为负数或None）
        e: 结束索引（可为负数或None）

    Returns:
        dict: 每个指标的平均值，如 {"test mrr": avg, "test auroc": avg, "test auprc": avg}
    """
    with open(json_file_path, "r") as f:
        results = json.load(f)

    metrics = {"test mrr": [], "test auroc": [], "test auprc": []}

    for result in results:
        for key in metrics.keys():
            val = _extract_first_float(result.get(key, ""))
            if val is not None:
                metrics[key].append(val)

    # 如果没有任何数据
    if not any(metrics.values()):
        print("❌ 未找到有效的指标数据")
        return None

    # 处理切片参数（基于 test mrr 的长度作为参考，如果需要也可改为更严格的检查）
    ref_len = (
        len(metrics["test mrr"])
        if metrics["test mrr"]
        else max(len(v) for v in metrics.values())
    )
    if s is not None:
        if s < 0:
            s = max(0, ref_len + s)
    else:
        s = 0

    if e is None:
        e = ref_len
    elif e < 0:
        e = max(0, ref_len + e)

    # 切片并计算统计量
    sliced = {k: v[s:e] for k, v in metrics.items()}

    results_summary = {}
    print("\n📊 统计结果:")
    print(f"考虑范围: [{s}, {e})")
    for k, vals in sliced.items():
        if vals:
            avg = sum(vals) / len(vals)
            std = (sum((x - avg) ** 2 for x in vals) / len(vals)) ** 0.5
            mx = max(vals)
            mn = min(vals)
            print(
                f"{k}: 平均 {avg:.4f} ± {std:.4f} (n={len(vals)})，max={mx:.4f}，min={mn:.4f}"
            )
            results_summary[k] = avg
        else:
            print(f"{k}: ❌ 无有效数据")
            results_summary[k] = None

    return results_summary


def calculate_stats_by_seed(json_file_path):
    """
    按照 seed 分组计算 test mrr, test auroc, test auprc 的统计数据
    """
    with open(json_file_path, "r") as f:
        results = json.load(f)

    seed_results = {}  # seed -> {metric: [values]}

    for result in results:
        seed = result.get("seed", 1)
        if seed not in seed_results:
            seed_results[seed] = {"test mrr": [], "test auroc": [], "test auprc": []}

        for key in seed_results[seed].keys():
            val = _extract_first_float(result.get(key, ""))
            if val is not None:
                seed_results[seed][key].append(val)

    print("\n📈 按Seed分组统计:")
    overall = {k: [] for k in ["test mrr", "test auroc", "test auprc"]}

    for seed, metrics_dict in seed_results.items():
        print(f"\nSeed {seed}:")
        for k, vals in metrics_dict.items():
            if vals:
                avg = sum(vals) / len(vals)
                std = (sum((x - avg) ** 2 for x in vals) / len(vals)) ** 0.5
                print(f"  {k}: {avg:.4f} ± {std:.4f} (n={len(vals)})")
                overall[k].extend(vals)
            else:
                print(f"  {k}: ❌ 无有效数据")

    print("\n🎯 总体统计:")
    overall_summary = {}
    for k, vals in overall.items():
        if vals:
            avg = sum(vals) / len(vals)
            std = (sum((x - avg) ** 2 for x in vals) / len(vals)) ** 0.5
            print(f"  {k}: {avg:.4f} ± {std:.4f} (n={len(vals)})")
            overall_summary[k] = (avg, std)
        else:
            print(f"  {k}: ❌ 无有效数据")
            overall_summary[k] = (None, None)

    return overall_summary

In [15]:
# exper_name = "hetero_sthn_rgfm"
# exper_name = "rgfm-htero"
exper_name = "rgfm-htero-newtrain"
# exper_name = "abl_no_rie"
# exper_name = "abl_sthn"
# exper_name = "abl_no_hetro"

# datasets = ["thgl-forum-subset", "thgl-github-subset","thgl-myket-subset","thgl-software-subset"]
datasets = ["thgl-forum-subset"]

for dataset in datasets:
    json_file_path = f"exper/{exper_name}/{dataset}/result/{dataset}_results.json"

    print(f"\n🔍 处理数据集: {dataset}")

    # 计算平均test mrr
    calculate_average_test_mrr(json_file_path, s=0, e=None)
    # calculate_average_test_mrr(json_file_path,s=None,e=None)


🔍 处理数据集: thgl-forum-subset

📊 统计结果:
考虑范围: [0, 10)
test mrr: 平均 0.8549 ± 0.0558 (n=10)，max=0.9439，min=0.8047
test auroc: 平均 0.8275 ± 0.0867 (n=10)，max=0.9665，min=0.7402
test auprc: 平均 0.5172 ± 0.1804 (n=10)，max=0.8460，min=0.3669
